## Imswitch/UC2-REST DEMO for 8P370 CBL Microscopy

**By Marcus Vroemen and Tom van Hattem**

*Using the open UC2 Rest and Imswitch Software*

This tutorial and demo will shows you how to get started with Imswitch and UC2 in Python. Imswitch is a software package that allows the creation of graphical user interfaces (GUIs) and interact with our hardware, the open UC2 Controller Board to turn on LED matrices or control the motors. The board itself will not handle cameras, these will usually be handled over USB. Most online tutorials will show you. how to flash the boards. This should already be done on the controller boards, but if not they can easily be reflashed with help of this page: https://youseetoo.github.io/ <br>
But before reflashing your computer will require drivers to see the controller board. If the official openUC2 boards are used with a ESP32-WROOM-32D controller and shield, you will need the CP210x drivers: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads
It is recommended to use Visual Studio Code or PyCharm, although Spyder is also possible (but please don't). <br>

IMPORTANT: Do not attempt to use the motors without an external power supply. Powering over USB will most certainly not work/damage your computer.

*Sidenote: also ensure that you are using a data cable. Charging micro usb cables will not be able to interact with the controller board. <br>*

Let's get started. First we need to configure the python environment


### Python Environment Configuration




1. Create a new Python environment called `8P370`. Do this within the Anaconda command prompt, run:


```python
conda create --name 8P370 python=3.8
```

2. Make sure to activate the `8P370` environment before proceeding to install pacakges with the following command:


```python
conda activate 8P370
```



3. Next, install `UC2-REST` and `ImSwitch`  packages (and other packages you might need at the place of <...>) with `pip` using the following command:

```python
pip install UC2-REST --user
pip install ImSwitchUC2==0.2.0.14
pip install ipykernel 
pip install <...> 
```

Hint: Make sure to only use `pip` from now on. Do not mix `pip` with `conda install`. Trust me.

###  Basic Python Code to control the microscope

To use python code to control the microscope, we need to communicate with the UC2 Controller board. This is a USB connection through COM ports. In Windows open the device manager, under Ports(COM&LPT) when the board is connected it should show `Silicon Labs CP210x USB to UART Bridge (COM4)`. COM4 could be a different port but by default it usually is 3 or 4. Change in the next code block the serialport to your COM port. It has to be a string.

In [1]:
%reload_ext autoreload 
%autoreload 2

#Import necessary libraries
import uc2rest as uc2
import numpy as np 
import time

In [2]:
"""Open communication with the UC2 Controller Board
#The log should print '{identifier_name:... etc.}'. If you restart the kernel please rerun this script
to ensure a com port connection has been established and is available to use. If only the state is printed 
it is likely that you have already run this code and the connection is still available in the kernel.
"""

serialport = "COM76" # for Windows - change accordingly
#serialport = "/dev/cu.SLAB_USBtoUART" # for MAC change accordingly
#serialport = "/dev/cu.wchusbserial110" # for MAC change accordingly

if 'ESP32' not in locals():
    ESP32 = uc2.UC2Client(serialport=serialport)
_state = ESP32.state.get_state()
print(_state)

Using API version 2
Attention, lasers are on channels 1,2,3
{'identifier_name': 'UC2_Feather', 'identifier_id': 'V2.0', 'identifier_date': 'Sep  7 202316:32:04', 'identifier_author': 'BD', 'IDENTIFIER_NAME': 'uc2-esp', 'configIsSet': 0, 'pindef': 'UC2_2', 'qid': 4}


# LED MATRIX

Now that we have initialized the COM port and UC2 controller we can test the LED matrix. Connect it to the LED1 port.

<img src="img//IMG20230926153841.jpg" alt="alt text" width="500"/>

In [14]:
# test LED
print("The LED pin is: "+str(ESP32.led.get_ledpin()))
time.sleep(2)
ESP32.led.send_LEDMatrix_full(intensity=(255, 255, 255))
time.sleep(2.5)
ESP32.led.send_LEDMatrix_full(intensity=(0, 0, 0))

The LED pin is: {'ledArrNum': 64, 'ledArrPin': 32, 'led_ison': 0, 'qid': 54, 'LEDArrMode': [0, 1, 2, 3, 4, 5, 6, 7]}


{'success': 1}

ClearCommError failed (PermissionError(13, 'Access is denied.', None, 5))


# Initializing and Moving the motors
Now lets connect the z-stage. Make sure to connect the external power supply, please don't blow up your usb ports. 

The following code snippets will help you moving the motors (XYZ) continously or at a known number of steps at a certain speed level (all measured in steps/s). <br>
**In general:** The axes are: 
A => 0
X => 1
Y => 2
Z => 3

The pin configuration for the uc2 board can be found at: https://youseetoo.github.io/

For the Z-stage:

<img src="img//IMG20230926161544.jpg" alt="alt text" width="350"/>

In [4]:
# we don't want to change the configuration now
# OR setup motors individually (according to WEMOS R32 D1)
if 0:
    ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 26, dirPin=16, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 2, position = 0, stepPin = 25, dirPin=27, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 17, dirPin=14, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 19, dirPin=18, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)


#ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 2, dirPin=33, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)
#ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 22, dirPin=21, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)
#ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 12, dirPin=14, enablePin=13, maxPos=None, minPos=None, acceleration=None, isEnable=1)

This pin configuration also can be reset to default values. For more information on the build in functions see this UC2-REST notebook: https://github.com/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client-PinConfigurator.ipynb



In [11]:
position1 = ESP32.motor.get_position(timeout=1)
print(position1)

ESP32.motor.move_z(steps=-10000, speed=5000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=5000, is_blocking=True)

ESP32.motor.move_z(steps=-10000, speed=10000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=10000, is_blocking=True)

ESP32.motor.move_z(steps=-10000, speed=100000, is_blocking=True)
ESP32.motor.move_z(steps=10000, speed=100000, is_blocking=True)

time.sleep(1) 

position2 = ESP32.motor.get_position(timeout=1)
print(position2)

[     0.      0.      0. -29300.]
[     0.      0.      0. -29300.]


The ESP32 object will keep an connection to the board open until you reset the kernel or disconnect the USB. To close the connection with the software please use the command below. 

In [20]:
ESP32.close()

A more extensive tutorial on the different features can be found in the official UC2-REST git: https://github.com/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb

# IMSWITCH

So far we have only used UC2-REST, but we can also use IMSWITCH. IMSWITCH provides a nice user interface. It is easiest to install a new environment. You can follow the commands below in anaconda cmd or in a build in environment (but the conda should be in PATH). The commands first clone the openuc2 fork of imswitch, then install a new environment based on the requirements from the fork, then downloads config files from Ben (one of the creators of openUC2). These config files should be located in your documents folder. 


```python
cd %HOMEPATH%\Documents
git clone https://github.com/openUC2/ImSwitch/
cd ImSwitch

conda create -n imswitch python=3.9 -y
conda activate imswitch
pip install -r requirements.txt --user

cd %HOMEPATH%\Documents
```


Note: if there is a folder called ImSwitchConfig => rename it!

```python	
git clone https://github.com/beniroquai/ImSwitchConfig
```

To start imswitch use this command in the cmd line.
```python
imswitch
```

# TROUBLESHOOTING
Once the `ESP32 = uc2.UC2Client(serialport=serialport)` command is executed, within the current kernel, the ESP32 class is available. To close this the port needs to be closed with .close(). If you reattept to run the command again, it will provide a error such as: 
```python
PermissionError(13, 'Access is denied.', None, 5)
``` 
As two instances are not possible. Please restart the kernel to proceed. If anything else goes wrong, also just restart the kernel. For the permission error, it is also likely that another program or kernel is running and has the com port opened. For example a program such as Ultimaker CURA or Arduino IDE is likely to interfere with UC2, therefore close it and disable opening the program on startup in the task manager (for convenience). 

```python
FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
``` 
For a not found error it is likely that the COM port is wrong (or usb not connected properly). 

There are several things that limit the board from being connected correctly to your laptop/pc. The first thing to check is if you have the correct drivers installed: https://www.silabs.com/developers/usb-to-uart-bridge-vcp-drivers?tab=downloads

Next check if you are using the correct cable, and if it shows up in the device manager. 



Also check that you are using the correct com port (as seen in the device manager.)

To check if the hardware works correctly it is also possible to check using https://youseetoo.github.io/indexWebSerialTest.html. This will ask in the browser to connect to the UC2 controller board and if a Z-stage motor or LED ring is correctly connected to the right port (labelled on the pcb) then you should be able to control them. 

Sometimes `pip UC2-REST --force-reinstall --user` will cause the code to execute properly. Feel free to try this. Remember to first activate your env 8P370 in anaconda (or other env manager).


In [ ]:
#Try to manually open the serial port to check if this works. 
# import serial
# import time
# c = serial.Serial('COM76', 9600)
# counter = 0
# while True:
#     signal = c.read()
#     print("running")
#     print(signal)
#     time.sleep(0.1)
#     c.flushOutput()
#     counter +=1
#     if counter == 10:
#         c.close()
#         break